# Logistic regression classifier

## imports, factorization of features, dataset preparation

In [1]:
import pandas as pd 
import numpy as np
%matplotlib inline 
import matplotlib.pyplot as plt
from tqdm import tqdm

def sigmoid(z): 
    return 1 / (1 + np.exp(-z))

def softmax(zs):
    return np.exp(zs) / sum(np.exp(zs))

In [2]:
# Read the split dataset
# Available on github/data/splits
df_train = pd.read_csv('bpi2017_train.csv')
df_test = pd.read_csv('bpi2017_test.csv')
df_val = pd.read_csv('bpi2017_val.csv')

In [3]:
df_train.head()

,Unnamed: 0,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID
0,0,Created,User_1,A_Create Application,Application,Application_1691306052,complete,2016-01-01 10:16:11.500000+00:00,Home improvement,New credit,Application_1691306052,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,statechange,User_1,A_Submitted,Application,ApplState_284636842,complete,2016-01-01 10:16:11.549000+00:00,Home improvement,New credit,Application_1691306052,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Created,User_1,W_Handle leads,Workflow,Workitem_831373279,schedule,2016-01-01 10:16:11.740000+00:00,Home improvement,New credit,Application_1691306052,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1299098074,withdraw,2016-01-01 10:17:31.573000+00:00,Home improvement,New credit,Application_1691306052,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Created,User_1,W_Complete application,Workflow,Workitem_1703931302,schedule,2016-01-01 10:17:31.584000+00:00,Home improvement,New credit,Application_1691306052,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# factorization of categorical atrtibutes (of interest) of the training data
df_train_f = df_train.copy()
vals, code_Action = pd.factorize(df_train_f['Action'])
df_train_f['Action'] = vals

vals, code_Origin = pd.factorize(df_train_f['EventOrigin'])
df_train_f['EventOrigin'] = vals

vals, code_lifecycle_transition = pd.factorize(df_train_f['lifecycle:transition'])
df_train_f['lifecycle:transition'] = vals

vals, code_loan_goal = pd.factorize(df_train_f['case:LoanGoal'])
df_train_f['case:LoanGoal'] = vals

vals, code_appl_type = pd.factorize(df_train_f['case:ApplicationType'])
df_train_f['case:ApplicationType'] = vals

# vals, code_concept_name = pd.factorize(df_train_f['concept:name'])
# df_train_f['concept:name'] = vals

# and for test data
df_test_f = df_test.copy()
vals, code_Action = pd.factorize(df_test_f['Action'])
df_test_f['Action'] = vals

vals, code_Origin = pd.factorize(df_test_f['EventOrigin'])
df_test_f['EventOrigin'] = vals

vals, code_lifecycle_transition = pd.factorize(df_test_f['lifecycle:transition'])
df_test_f['lifecycle:transition'] = vals

vals, code_loan_goal = pd.factorize(df_test_f['case:LoanGoal'])
df_test_f['case:LoanGoal'] = vals

vals, code_appl_type = pd.factorize(df_test_f['case:ApplicationType'])
df_test_f['case:ApplicationType'] = vals

vals, code_concept_name = pd.factorize(df_test_f['concept:name'])
# df_test_f['concept:name'] = vals

# to see the "code" - so which number corresponds to which class - print the code_... for the attribute of interest

In [5]:
# select the features that will be used for prediction
features = ['Action', 'EventOrigin', 'lifecycle:transition', 'case:LoanGoal', 'case:ApplicationType', 'case:RequestedAmount']

In [6]:
# split the predictors and the outcome variable
X_train = df_train_f[features]
y_train = df_train_f['concept:name']
X_test = df_test_f[features]
y_test = df_test_f['concept:name']

In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

# initialize and use the scaler
standardizer = StandardScaler()
X_train_st = standardizer.fit_transform(X_train)
X_test_st = standardizer.transform(X_test)

# fit the model
lr = LogisticRegression(multi_class='multinomial')
lr.fit(X_train_st, y_train);

# Obtain decision function for first training unit
# lr.decision_function(X_train_st[0])

/Users/hannabroszczak/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [60]:
# obtain the prediction for the 1st unit:
prediction = np.argmax(softmax(lr.decision_function(X_train_st[0].reshape(1,-1))[0]).round(1))
code_concept_name[prediction]

'A_Concept'

In [82]:
# all_predictions = np.argmax(softmax(lr.decision_function(X_train_st[15].reshape(1,-1))[0]).round(1))
prediction = np.argmax(softmax(lr.decision_function(X_train_st[15].reshape(1,-1))[0]).round(1))
prediction

24

In [84]:
lr.predict_proba(X_train_st[:2]).round(3)

array([[0.001, 0.001, 0.001, 0.001, 0.99 , 0.001, 0.001, 0.001, 0.001,
        0.001, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ],
       [0.148, 0.052, 0.147, 0.148, 0.001, 0.02 , 0.106, 0.076, 0.124,
        0.178, 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ]])

In [11]:
# evaluate the model (??)
lr.score(X_test_st,y_test)

0.30840242458022715

## Add the predictions to the initial dataframe

In [19]:
# initialize column for predictions
df_train['predicted_event'] = 0

In [24]:
# to "block" the printing of warnings:
import warnings
warnings.filterwarnings(action='ignore')

In [25]:
# add predictions to the column in df_train
for x in tqdm(range(0, len(X_train))):
    df_train['predicted_event'].loc[x] = np.argmax(lr.decision_function(X_train_st[x].reshape(1,-1))[0])

100%|█████████████████████████████████| 621131/621131 [01:46<00:00, 5822.99it/s]


In [39]:
df_train['predicted_event'].unique()

array([ 4,  9, 24, 19, 12, 13, 21, 20,  6])

In [40]:
# make a dictionary to translate the factors into initial event names
keys = [i for i in range(0, len(code_concept_name))]
codes = code_concept_name
dict_concept_name = {keys[i]: codes[i] for i in range(len(codes))}

In [41]:
# translate the factors into initial event names
df_train['predicted_event_name'] = X_train['predicted_action'].map(dict_concept_name)

In [47]:
len(df_train[df_train['concept:name']==df_train['predicted_event_name']])/len(df_train)

0.02534409005507695

### to be investigated further - hyperparam. tuning:

multi_class type (e.g. multi_class='ovr')

penalty (lasso)